In [ ]:
import configparser
import json
import asyncio
from datetime import date, datetime

from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (
    PeerChannel
)


# some functions to parse json date
class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()

        if isinstance(o, bytes):
            return list(o)

        return json.JSONEncoder.default(self, o)


# Reading Configs
config = configparser.ConfigParser()
config.read("config-bastiaugen.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']

api_hash = str(api_hash)

phone = config['Telegram']['phone']
username = config['Telegram']['username']

# Create the client and connect
client = TelegramClient(username, api_id, api_hash)


async def main(phone):
    async with client:
        await client.start()
        print("Client Created")

        # Check if the user is authorized, otherwise initiate the sign-in process
        if not await client.is_user_authorized():
            await client.send_code_request(phone)
            try:
                await client.sign_in(phone, input('Enter the code: '))
            except SessionPasswordNeededError:
                await client.sign_in(password=input('Password: '))

        me = await client.get_me()

        # Reading channels from a text file
        with open('input_nazi.txt', 'r') as file:
            channels = [line.strip() for line in file]
            
        channel_data = {}  # Dictionary to store the channel data
        
        # Iterate over the channels
        for channel in channels:
            if channel.isdigit():
                entity = PeerChannel(int(channel))
            else:
                entity = channel

            try: 
                my_channel = await client.get_entity(entity)
            except ValueError: 
                print("error")
                continue

            offset_id = 0
            limit = 100
            filtered_messages = []
            total_messages = 0
            total_count_limit = 0

            # Retrieve channel history
            while True:
                print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
                history = await client(GetHistoryRequest(
                    peer=my_channel,
                    offset_id=offset_id,
                    offset_date=None,
                    add_offset=0,
                    limit=limit,
                    max_id=0,
                    min_id=0,
                    hash=0
                ))

                if not history.messages:
                    break

                messages = history.messages
            
                # Filter messages and extract mentioned channels
                for message in messages:
                    if message.message and "https://t.me/" in message.message:
                        words = message.message.split()
                        filtered_words = [word for word in words if word.startswith("https://t.me/")]
                        filtered_string = " ".join(filtered_words)
                        filtered_messages.append(filtered_string)
                        
                # Store the filtered messages in the channel data dictionary
                main_channel = channel
                channel_data[main_channel] = filtered_messages

                offset_id = messages[len(messages) - 1].id
                total_messages = len(filtered_messages)
                if total_count_limit != 0 and total_messages >= total_count_limit:
                    break
            

        # Save the channel data to a JSON file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{timestamp}_channel_data.json"

        with open(filename, "w") as outfile:
            json.dump(channel_data, outfile, cls=DateTimeEncoder)

await main(phone)

# then iterating over the mentions in those json/csv file, thus creating a new file with the same structrue as the previous one